Next work:Connect extra data source / file for ChatGPT

In [17]:
pip install openai

In [ ]:
from google.colab import userdata
userdata.get('API_KEY')

In [19]:
from openai import OpenAI
client = OpenAI(api_key = userdata.get('API_KEY'))

Step1 : Create assistant

In [20]:
assistant = client.beta.assistants.create(
    name="World Cuisine Recipes Recommender",
    instructions= "You are a world food recipes recommender. When a user asks about a dish, provide details including the food name, time to prepare, steps,any tags and ingredients needed.",
    tools=[{"type": "file_search"}],
    model="gpt-3.5-turbo" ,
)


In [21]:
vector_store = client.beta.vector_stores.create(name="Food recipe recommender")

# Ready the files for upload to OpenAI
file_paths = ["101_Square_Meals.pdf","recipes.json"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


In [22]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [23]:
def create_message_and_run(client, assistant, user_message, thread=None):
    # Create a new message in the thread and start a new run
    if not thread:
        thread = client.beta.threads.create()
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role= "user",
        content= user_message
        )
    run = client.beta.threads.runs.create(
    thread_id=thread.id, assistant_id=assistant.id
    )
    return run, thread


In [24]:
user_message = "Now i need you some advices for food."
run, thread = create_message_and_run(client,assistant,user_message)

In [25]:
def submit_tool_outputs(run,thread):
    run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id
    )
    return run

In [26]:
while True:
    # Retrieve the current run status
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run.status == "requires_action":
        run = submit_tool_outputs(run,thread)
        continue

    if run.status == "completed":
        # Retrieve messages when the run is completed
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print("\n"+latest_message.role+": "+text)
        # Prompt for user input to continue or stop the conversation
        user_input = input()
        if user_input.lower() == "stop":
            break
        # Create a new message and poll for a new run based on the user's response
        run,thread = create_message_and_run(client,assistant=assistant,user_message=user_input,thread=thread)
        continue



assistant: Of course! I can provide you with some food advice. Could you please specify if you are looking for cooking tips, recipe recommendations, dietary advice, or any particular food-related topic you would like to learn more about?
Now I have chicken and mushroom, I want to cook some food with no more than 1 hour.

assistant: Here are a couple of recipes that you can prepare with chicken and mushrooms in under 1 hour:

1. **Chicken Curry**:
   - **Ingredients**: Cooked chicken, button mushrooms, pepper, onion, peas, carrot, garlic, curry paste, vegetable oil, chicken stock.
   - **Preparation Time**: 5 minutes
   - **Cooking Time**: 15 minutes
   - **Method**: Slice and cook vegetables and garlic, add mushrooms and pepper, stir in curry paste, cooked chicken, peas, and stock. Simmer for 10 minutes and serve with rice【6:0†source】.

2. **Chicken in Tomato Sauce**:
   - **Ingredients**: Onion, carrot, garlic, parsley, vegetable oil, chicken breast fillets, passata or tomato-based p